In [25]:
import requests
import json
from datetime import datetime
import os


In [22]:
JIRA_URL = "https://mdr21.atlassian.net/"
JIRA_EMAIL = "abuyehia100@gmail.com"
API_TOKEN = "ATATT3xFfGF09UtsyqZ7HzlPDKVPU3qVUnkVL3sss6I6XaYGWf10Mg8y3QSlJ9XlDTeGLf_V54_RZugWewygmR2lW1U_Dm8AzFUGweywHvbV8Yk04eey8iM5YXE3unfwNtT_qbSiJuYOZg-nQnMmFmVjpYM-VyXr7wk4Bmmjd-gbd3dqE9nMFss=7028947F"
PROJECT_KEY = "MY Scrum Project"

In [23]:
def get_text_from_description(description_obj):
    """
    Extracts plain text from Jira's Atlassian Document Format (ADF) description.
    """
    if not description_obj or 'content' not in description_obj:
        return ""
    
    text_parts = []
    for content_block in description_obj['content']:
        if 'content' in content_block:
            for text_item in content_block['content']:
                if 'text' in text_item:
                    text_parts.append(text_item['text'])
    return "\n".join(text_parts)

In [29]:
search_url = f"{JIRA_URL}/rest/api/3/search"
auth = (JIRA_EMAIL, API_TOKEN)
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}
jql_query = f'project = "{PROJECT_KEY}"'

all_issues_raw = []
start_at = 0
max_results = 100
requested_fields = [
    'summary', 'description', 'status', 'issuetype', 'reporter', 'assignee',
    'components', 'created', 'labels', 'priority', 'project'
]

print(f"Fetching issues from project: {PROJECT_KEY}...")

while True:
    params = {
        'jql': jql_query,
        'startAt': start_at,
        'maxResults': max_results,
        'fields': ','.join(requested_fields)
    }
    try:
        response = requests.get(
            search_url, headers=headers, params=params, auth=auth, timeout=30
        )
        response.raise_for_status()
        data = response.json()
        issues_on_page = data.get('issues', [])
        all_issues_raw.extend(issues_on_page)
        total_issues = data.get('total', 0)
        
        if start_at + len(issues_on_page) >= total_issues:
            break
        start_at += len(issues_on_page)
        print(f"Fetched {len(all_issues_raw)} of {total_issues} issues...")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        break

print("-" * 30)
print(f"Total issues fetched: {len(all_issues_raw)}")
print("Transforming data into the desired structure...")
print("-" * 30)

# --- Transformation Logic ---
transformed_issues_list = []
for issue in all_issues_raw:
    fields = issue.get('fields', {})
    
    # Safely get the assignee's display name, or None if not assigned
    assignee_obj = fields.get('assignee')
    assignee_name = assignee_obj['displayName'] if assignee_obj else None

    # Safely get the created date and reformat it
    created_date_str = fields.get('created')
    if created_date_str:
        # Parse the timestamp with timezone, then format it
        created_datetime = datetime.strptime(created_date_str, "%Y-%m-%dT%H:%M:%S.%f%z")
        formatted_created_date = created_datetime.strftime("%Y-%m-%d %H:%M:%S")
    else:
        formatted_created_date = None

    transformed_issue = {
        "_id": issue.get('id'),
        "assignee": assignee_name,
        "components": fields.get('components', []),
        "created": formatted_created_date,
        "description": get_text_from_description(fields.get('description')),
        "issuetype": fields.get('issuetype'),
        "labels": fields.get('labels', []),
        "priority": fields.get('priority'),
        "projectname": fields.get('project', {}).get('name'),
        "summary": fields.get('summary')
    }
    transformed_issues_list.append(transformed_issue)

# --- Final Output ---

# Print the final list to the console
print(json.dumps(transformed_issues_list, indent=2))
output_folder = "deploy and test/jsons"
output_filename = "jira_issue.json"
output_path = os.path.join(output_folder, output_filename)
os.makedirs(output_folder, exist_ok=True)
# Save the transformed data to a file
# with open(output_path, 'w') as f:
#     json.dump(transformed_issues_list, f, indent=2)

print("\nTransformed issue data saved to jira_issues_transformed.json")

Fetching issues from project: MY Scrum Project...
------------------------------
Total issues fetched: 1
Transforming data into the desired structure...
------------------------------
[
  {
    "_id": "10034",
    "assignee": "Mohammad Alomar",
    "components": [],
    "created": "2025-06-18 16:26:22",
    "description": "After successfully installing a cluster, I saved YARN configs (changed only 1 property). YARN service would not start with ResourceManager saying it cannot bind to port.\\n\\nTurns out that certain properties in {{yarn-site.xml}} were set to null value.\\n{noformat}\\n\\\"yarn.resourcemanager.admin.address\\\" : \\\"null\\\",\\n\\\"yarn.resourcemanager.resource-tracker.address\\\" : \\\"null\\\",\\n\\\"yarn.resourcemanager.scheduler.address\\\" : \\\"null\\\",\\n\\\"yarn.resourcemanager.address\\\" : \\\"null\\\",\\n\\\"yarn.log.server.url\\\" : \\\"null\\\",\\n{noformat}\\nSimilar problem with MR2 also. The {{mapred-site.xml}} had 19 properties which were null.\\n\\

In [ ]:
"https://mdr21.atlassian.net/browse/SCRUM-3?atlOrigin=eyJpIjoiYTE5YjQ4M2NiNThmNGRmY2I3ODJhZDA0OGJiN2MxMjYiLCJwIjoiaiJ9"